## An Analysis of the 2024 Syracuse Mets
In this analysis, I will examine basic statistics scraped from Baseball Reference as well as advanced sabremetrics from Baseball Savant's Minor League Statcast database. The focus will be on hitters with over 100 plate appearances to ensure an accurate understanding of players who have a decent sample size of play in AAA. Additionally, I’ll concentrate on players under the age of 27, as the Mets organization has emphasized their young prospects who can contribute to the team's near future success.

In [1]:
import pandas as pd
from utils import *

# You can also prevent truncation of large DataFrames.
pd.set_option('display.expand_frame_repr', False)

# Set maximum column width.
pd.set_option('display.max_colwidth', 200)

# Set the maximum number of columns displayed.
pd.set_option('display.max_columns', None)

# Set the maximum number of rows displayed.
pd.set_option('display.max_rows', None)

# Set display width to ensure DataFrame fits in the notebook.
pd.set_option('display.width', 3000)

# Select Mets players below 27 and with over 100 plate appearences.
completeMetsAAAStats = completeMetsAAAStats[(completeMetsAAAStats['PA'] > 100) & (completeMetsAAAStats['Age'] <= 26)]

## Initial Analysis / Basic Slash Line

To begin our analysis, we’ll start with the traditional slash line—batting average, on-base percentage, and slugging percentage. Although OPS (On-base Plus Slugging) isn't traditionally included in the slash line, I've added it because it's one of the most effective non-advanced stats for gauging a hitter's overall performance.

Based on our criteria, five players meet the requirements: Brett Baty, Mark Vientos, Luisangel Acuña, Rhylan Thomas, and Drew Gilbert.

Among these players, Brett Baty and Mark Vientos are the standouts, each posting an OPS above .800, suggesting they possess both power and the ability to get on base, at least from a surface-level perspective.

Next, we’ll compare how these players' stats stack up against AAA averages.

In [2]:
# Select the basic statline stats and basic info I want to see for each player: Player, Age, G, PA, AB, BA, OBP, SLG, OPS
print(completeMetsAAAStats[defaultInfo + slashLine].sort_values(by='OPS', ascending=False))

              Player   Age    G   PA   AB     BA    OBP    SLG    OPS
3        Baty, Brett  24.0   56  246  211  0.261  0.358  0.536  0.885
27     Vientos, Mark  24.0   31  132  116  0.284  0.371  0.500  0.876
0   Acuna, Luisangel  22.0  110  486  452  0.265  0.313  0.369  0.676
25    Thomas, Rhylan  24.0   49  192  179  0.229  0.281  0.374  0.663
9      Gilbert, Drew  23.0   26  111   97  0.216  0.315  0.309  0.577


## League Averages Comparison
<p>
In terms of batting average, Brett Baty sits just above the league mean. However, looking further down his stat line, it’s clear he's far from average. Baty ranks in the top half of the league for OBP, a critical metric in today’s game where getting runners on base is key to scoring. However, his true strength lies in his power. Baty's slugging percentage is comfortably in the top quartile of the league, as is his OPS. This combination highlights his ability to hit for power without completely sacrificing his on-base skills.

Next, let’s consider Mark Vientos. Although he’s been promoted to the Majors, it's valuable to analyze the stats of players who have successfully transitioned to the MLB. Vientos’ 2024 season has been amazing so far, and his AAA stats before being called up reflect this. His batting average ranked comfortably in the upper quartile, and this trend holds true across most of his slash line stats, except for OBP, where he falls into the third quartile. Like Baty, Vientos shows strong power-hitting ability, but he slightly edges out Baty in terms of getting on base through contact. However, Baty retains a slight edge in power if this is purely based on slugging.

Now, let’s take a closer look at one of our top prospects: Luisangel Acuña. At first glance, his slash line is quite underwhelming. He ranks in the bottom quartile for all stats except batting average, where he falls into the third quartile. This suggests that while Acuña can get on base through hits, his overall ability to get on base and power are lacking.

Many of these observations for Acuña also apply to Drew Gilbert. Like Acuña, Gilbert ranks in the bottom quartile for all slash line stats, including OBP. However, it's important to consider his limited experience at the AAA level—he's barely had 100 plate appearances and likely needs more time to adjust especially after injury. Gilbert previously put up solid numbers at all lower levels, so there's huge potential for improvement.

We traded Rylan Thomas for Ryne Stanek so I won't focus on him too much.
</p>

In [3]:
# Clean and convert columns that are mistyped.
columnsToConvert = ['OBP', 'SLG', 'BA']
leaguewideAAAStatsDf = convertColumnsToNumeric(leaguewideAAAStatsDf, columnsToConvert).copy()

# Calculate OPS for leaguewide data, this isn't a stat provided to us by Baseball Savant in the league data.
leaguewideAAAStatsDf['OPS'] = leaguewideAAAStatsDf['OBP'] + leaguewideAAAStatsDf['SLG']

# Select all players in AAA with over 100 plate appearences.
leaguewideAAAStatsDf = leaguewideAAAStatsDf[leaguewideAAAStatsDf['PA'] > 100]

# Get league mean, standard deviation, percentiles, and min/max values for BA, OBP, SLG, and OPS and concat the dataframes for easier viewing.
summary_df = pd.concat([
    leaguewideAAAStatsDf['BA'].describe(),
    leaguewideAAAStatsDf['OBP'].describe(),
    leaguewideAAAStatsDf['SLG'].describe(),
    leaguewideAAAStatsDf['OPS'].describe(),
], axis=1)

print(summary_df)

               BA         OBP         SLG         OPS
count  471.000000  471.000000  471.000000  471.000000
mean     0.257062    0.347420    0.428486    0.775907
std      0.038070    0.042097    0.081750    0.112078
min      0.135000    0.204000    0.226000    0.431000
25%      0.234000    0.322000    0.374000    0.703000
50%      0.258000    0.346000    0.431000    0.780000
75%      0.280000    0.373000    0.480500    0.853500
max      0.401000    0.515000    0.678000    1.179000


## Actual Stats vs Expected Stats
Before diving into the potential reasons behind these players' performances, it's crucial to assess how they're performing relative to their expected stats. Expected stats help provide a metric that considers the quality of contact (e.g., exit velocity, launch angle) to estimate what a player's performance should look like under neutral conditions.

Let's examine Brett Baty. 
Baty's actual stats are lower than his expected stats across all categories. This discrepancy doesn't necessarily indicate underperformance. While it’s possible he's underachieving, a more likely explanation is that Baty has been somewhat unlucky, with hard-hit balls not finding gaps or encountering tough defensive alignments. Given that his xBA sits at .278, still a strong figure, it suggests that if Baty continues to produce quality at-bats and solid contact, we should expect his actual batting average to rise. A similar analysis applies to his OBP vs. xOBP, SLG vs. xSLG, and wOBA vs. xwOBA. Sustaining this level of quality should lead to an upward trend in his slash line.

Now let's move on to Mark Vientos. 
Vientos presents an interesting case. Although he's currently outperforming Baty in BA and OBP and is only slightly behind in SLG, his expected stats are lower than Baty’s across the board. This suggests that Vientos may be benefiting from some luck, such as favorable bounces or bloop hits, or perhaps hitting home runs and extra-base hits that might not always clear the fence or fall in for hits. While this raises questions about the sustainability of his current stats, I peronsally believe these should be taken with a grain of salt. It's essential to consider the quality of contact and power Vientos is generating, which we’ll analyze further later on.

Acuña, like Vientos, is outperforming his expected stats, yet both his actual and expected stats are lower than Vientos’. This implies that while Acuña might be experiencing a similar level of good fortune, he’s not seeing the same payoff, evidenced by his lower average, on-base percentage, and power output. The observations regarding Vientos' contact quality and potential luck also apply to Acuña and really to all prospects ranked beneath him.

While these stats are valuable for monitoring and analysis, they may not directly translate to concrete future performance. Despite Baty's better expected stats compared to Vientos, he has struggled at the major league level, whereas Vientos has found success this season. This is not to say ignore these stats, but these metrics are more descriptive than predictive and should be interpreted with caution.

In [4]:
print(completeMetsAAAStats[defaultInfo + actualVsEx + ['OPS']].sort_values(by='OPS', ascending=False).to_string())

              Player   Age    G   PA   AB     BA    xBA    OBP   xOBP    SLG   xSLG   wOBA  xwOBA    OPS
3        Baty, Brett  24.0   56  246  211  0.261  0.278  0.358  0.375  0.536  0.552  0.381  0.397  0.885
27     Vientos, Mark  24.0   31  132  116  0.284  0.237  0.371  0.331  0.500  0.444  0.376  0.337  0.876
0   Acuna, Luisangel  22.0  110  486  452  0.265  0.236  0.313  0.287  0.369  0.325  0.299  0.273  0.676
25    Thomas, Rhylan  24.0   49  192  179  0.229  0.203  0.281  0.257  0.374  0.281  0.286  0.242  0.663
9      Gilbert, Drew  23.0   26  111   97  0.216  0.189  0.315  0.291  0.309  0.282  0.286  0.264  0.577


# Batter Profiles

## Brett Baty
Let's take a look back at the profile we've developed for Brett Baty before proceeding. Brett Baty is someone who can get on base at an above average rate but his strength lies in his slugging. He's excellent at hitting for power. However, after looking at his expected stats, it seems like he's getting unlucky on some hard hit balls or defensive alignments, leading to lower than expected hitting stats. But if he continues to hit like he is right now, his slash line should go up. To confirm this I'll look at BABIP, Hard Hit%, hit distance and launch angle, as well as both of his barrel ratios and exit velos. 

In terms of BABIP, Baty falls into the bottom quartile meaning that whenever he puts the ball in play, he's getting on base at one of the lowest rates in the league. Now a low BABIP can be an indicator for a number of things. Bad luck, weak contact, lots of shift adjusments, or even slow on the bases. Looking at some other stats available, we can take out the possibility of weak contact. Baty's Hard Hit%, adjusted exit velo, hit distance, and barrel ratios are all in the top quartile. Meaning he hits the ball hard at one of the highest rates in the league. For non adjusted exit velo he also still falls in the top half of the league, just missing the cut off for the upper quartile. This almost perfectly fits the description we came up with for Baty. He hits the ball extremely hard, but on balls in play he can get unlucky, hit into the shift, or he's slow on the base path. 

Luckily, with Baseball Savant's minor league statcast search, we can look at pitch-by-pitch statistics.

In [5]:
# Determine columns that are getting cleaned and converted
columnsToConvert = ['BABIP', 'Hard Hit%', 'EV (MPH)', 'Adj. EV (MPH)', 'Dist (ft)', 'LA (°)', 'Barrel/BBE%', 'Barrel/PA%']
leaguewideAverages = describeColumns(leaguewideAAAStatsDf, columnsToConvert)

print(leaguewideAverages)
print(completeMetsAAAStats[defaultInfo + contactQuality].sort_values(by='Barrel/BBE%', ascending=False).iloc[0].to_string())

            BABIP   Hard Hit%    EV (MPH)  Adj. EV (MPH)   Dist (ft)      LA (°)  Barrel/BBE%  Barrel/PA%
count  471.000000  471.000000  471.000000     471.000000  471.000000  471.000000   471.000000  471.000000
mean     0.312605   35.555626   87.719108      91.218896  154.777070   13.382166     6.336518    3.971550
std      0.044797    8.125097    2.313921       0.818278   11.143512    4.839380     3.711165    2.194024
min      0.153000    9.400000   80.400000      89.200000  126.000000   -3.000000     0.000000    0.000000
25%      0.286000   30.600000   86.100000      90.600000  147.000000   10.000000     3.400000    2.300000
50%      0.314000   35.800000   87.700000      91.200000  155.000000   13.000000     5.700000    3.700000
75%      0.340000   41.450000   89.400000      91.700000  162.000000   17.000000     8.800000    5.400000
max      0.486000   59.100000   95.500000      94.100000  184.000000   26.000000    18.000000   10.700000
Player           Baty, Brett
Age              

Unfortunately, I can't get descriptive league wide stats on how many hard hit outs each hitter has due to some Baseball Savant limitations. If I try to get league wide pitch by pitch stats, the pitches only go back to mid-July and I'd have to scrape the rest of the dates/pitches dynamically which posed its own set of problems. Ultimately, it seemed like more work then necessary for something I would use once. However, in the future as I expand this project I will be looking to solve this issue.

I was able to do a similar search using Baseball Savant's filters and found out Baty was in the bottom half of hitters with barrel/hard hit outs. So it doesn't seem like he's getting anymore unlucky than his peers.

Moving on to the location of his outs, we find why Baty's BABIP and BA is lower than expected: he's hitting defensive shifts. So far this season he's had 105 outs in play and 42 of these were to the first basemen or second basemen, which is about 40%. If we expand this to include the shortstop who is very often involved in shifts, it would go up to 58 outs into the shift, or 55% of Baty's in-play outs. 

In [6]:
# Modify the column name for pitch type
metsAAApbpSavant.columns.values[0] = 'pitch_type'

# Clean and convert 
colToCon = ['launch_speed']
batyPbp = convertColumnsToNumeric(metsAAApbpSavant, colToCon)

# Select info for Brett Baty
batyPbp = batyPbp[(batyPbp['player_name'] == 'Baty, Brett')]

# Select pitches were the outcome was an out in play
batyPbpOuts = batyPbp[batyPbp['events'].isin(pbpOuts)].copy()

# Select outs that were a barrel or hard hit outs Baty Had
hardHitOuts = batyPbpOuts[(batyPbpOuts['player_name'] == 'Baty, Brett') & (batyPbpOuts['launch_speed'] >= 95)].copy()
barrelOuts = batyPbpOuts[hardContact][(batyPbpOuts['launch_speed'] >= 95) & (batyPbpOuts['launch_angle'] >= 26)].copy()

print('Hard hard outs: ' + str(len(hardHitOuts)))
print('Barrel outs: ' + str(len(barrelOuts)) + '\n')
print(len(batyPbpOuts))
print(batyPbpOuts['hit_location'].value_counts().to_string())

Hard hard outs: 28
Barrel outs: 4

105
4.0    26
3.0    16
6.0    16
7.0    15
5.0    10
8.0    10
9.0     5
1.0     4
2.0     3


## Mark Vientos

Moving on to Mark Vientos, we've established he can hit for average pretty well and he was getting on base and slugging at pretty good rates too. However, his expected stats suggest that he might be getting lucky for the quality of his contact. Maybe a few lucky bounces, bloop hits, or some extra base hits/home runs that wouldn't normally fall for hits. To examine why he's out peforming his expected stats, I'll look at the same set of statistics as I did Baty.

Looking at Vientos' BABIP, we see it's extremely high and falls well into the top quartile of the league. This could mean a few things. It could in fact mean that he's getting lucky with his contact and some balls are falling, but it could also mean whenever he's putting the ball in play it's solid contact. To state the obvious, harder hit balls are more likely to turn into hits. A high BABIP can also be an indicator for a player that's great at beating the shift or doesn't get shifted on very much. Looking at some of Vientos' other quality of contact stats, we can see why he maintains his high BABIP. He's making great contact with the ball. He falls into the top quartile of every category except hit distance where he falls into the bottom quartile, likely the result of his low launch angle which also falls into the last quartile. My guess is this is why his expected stats are lower than his actual stats. Expected statistics are calculated using a combination of exit velocity, launch angle and, on certain types of batted balls, sprint speed. So a launch angle as low as Vientos' would definitely bring down his expected stats.

In [7]:
print(leaguewideAverages)
print(completeMetsAAAStats[defaultInfo + contactQuality].sort_values(by='Barrel/BBE%', ascending=False).iloc[1].to_string())

            BABIP   Hard Hit%    EV (MPH)  Adj. EV (MPH)   Dist (ft)      LA (°)  Barrel/BBE%  Barrel/PA%
count  471.000000  471.000000  471.000000     471.000000  471.000000  471.000000   471.000000  471.000000
mean     0.312605   35.555626   87.719108      91.218896  154.777070   13.382166     6.336518    3.971550
std      0.044797    8.125097    2.313921       0.818278   11.143512    4.839380     3.711165    2.194024
min      0.153000    9.400000   80.400000      89.200000  126.000000   -3.000000     0.000000    0.000000
25%      0.286000   30.600000   86.100000      90.600000  147.000000   10.000000     3.400000    2.300000
50%      0.314000   35.800000   87.700000      91.200000  155.000000   13.000000     5.700000    3.700000
75%      0.340000   41.450000   89.400000      91.700000  162.000000   17.000000     8.800000    5.400000
max      0.486000   59.100000   95.500000      94.100000  184.000000   26.000000    18.000000   10.700000
Player           Vientos, Mark
Age            

Let's take a closer look at Vientos' hits into play. Specifically, balls in play that were less than the league average launch angle. Since I can't look at the average amount of hits for players on balls below a 13 degree launch angle on a league wide scale due to data limitations, I'm just going to compare Vientos to the other top Mets prospects. Out of 46 balls hit into play, Vientos was able to turn 19 of these into hits. So about 41% of his contact where his launch angle was below average, he was able to turn into a hit. Compared to Acuna and Baty, this is well above their corresponding 35% and 26%.

Combining these with his contact quality stats, we can see Vientos is making hard contact and putting the ball in play which highly likely leads to him having a high BABIP. Additonally, I'd say his actual output is higher than his expected due to his low average launch angle and while he may be getting lucky on some of his hits, it's not important because he's still making great contact all around. Even on his hits into play below 13°, he still falls into the top half of the league's exit velocity. 

In [8]:
colToCon = ['launch_angle']
vientosPbp = convertColumnsToNumeric(metsAAApbpSavant, colToCon)

# Select info for Mark Vientos and Luisangel Acuna
vientosPbp = vientosPbp[vientosPbp['player_name'] == 'Vientos, Mark']
acunaPbp = metsAAApbpSavant[metsAAApbpSavant['player_name'].str.contains('Luisangel', case=False, na=False)]

# Select only balls hit into play with a launch angle below 13° for Vientos, Baty, and Acuna
vientosPbpHits = vientosPbp[(vientosPbp['description'] == 'hit_into_play') & (vientosPbp['launch_angle'] < 13)].copy()
batyPbpHits = batyPbp[(batyPbp['description'] == 'hit_into_play') & (batyPbp['launch_angle'] < 13)].copy()
acunaPbpHits = acunaPbp[(acunaPbp['description'] == 'hit_into_play') & (acunaPbp['launch_angle'] < 13)].copy()

# Create a DataFrame with the results
totalOutcomeResults = {
    'Vientos': vientosPbpHits['events'].value_counts(),
    'Baty': batyPbpHits['events'].value_counts(),
    'Acuna': acunaPbpHits['events'].value_counts()
}
df = pd.DataFrame(totalOutcomeResults)

# Add a total row for each player
df.loc['Total'] = [len(vientosPbpHits), len(batyPbpHits), len(acunaPbpHits)]

# Print the result data
print('Average launch speed on balls in play: ' + str(vientosPbpHits['launch_speed'].mean()) + '\n')
print(df)

Average launch speed on balls in play: 88.43478260869566

                           Vientos  Baty  Acuna
double                         2.0   2.0   10.0
double_play                    NaN   1.0    NaN
field_error                    2.0   1.0    8.0
field_out                     19.0  52.0  109.0
fielders_choice                1.0   NaN    NaN
force_out                      4.0   3.0   19.0
grounded_into_double_play      1.0   3.0   12.0
sac_bunt                       NaN   NaN    2.0
single                        17.0  19.0   72.0
Total                         46.0  81.0  232.0


# Luisangel Acuña
Assesing our profile on Acuna, we essentially saw he's above the league batting average indicating he can somewhat get on base using his bat. However, his OBP and SLG are underwhelming meaning his overall ability to get on base as well as his power is lacking. Similar to Vientos, his stats are well above his expected, indicating to me he is probably getting similarly lucky but seeing less of a payoff for it. 

Looking at his quality of contact statsitics more or less confirms these suspicions. For every metric, he's in the bottom half of the league meaning his contact is pretty poor too. And for the categories he's above average in, he just barely falls into the third quartile.

In [9]:
print(leaguewideAverages)
print(completeMetsAAAStats[defaultInfo + contactQuality].sort_values(by='Barrel/BBE%', ascending=False).iloc[3].to_string())

            BABIP   Hard Hit%    EV (MPH)  Adj. EV (MPH)   Dist (ft)      LA (°)  Barrel/BBE%  Barrel/PA%
count  471.000000  471.000000  471.000000     471.000000  471.000000  471.000000   471.000000  471.000000
mean     0.312605   35.555626   87.719108      91.218896  154.777070   13.382166     6.336518    3.971550
std      0.044797    8.125097    2.313921       0.818278   11.143512    4.839380     3.711165    2.194024
min      0.153000    9.400000   80.400000      89.200000  126.000000   -3.000000     0.000000    0.000000
25%      0.286000   30.600000   86.100000      90.600000  147.000000   10.000000     3.400000    2.300000
50%      0.314000   35.800000   87.700000      91.200000  155.000000   13.000000     5.700000    3.700000
75%      0.340000   41.450000   89.400000      91.700000  162.000000   17.000000     8.800000    5.400000
max      0.486000   59.100000   95.500000      94.100000  184.000000   26.000000    18.000000   10.700000
Player           Acuna, Luisangel
Age         

I didn't do this for Vientos or Baty because of their success at the plate, but for Acuña I think it's important to look at why he might potentially be underperforming. Looking at each his outs from this seaosn, there's a clear trend. He struggles most with velocity pitches like a fastball/sinker and has some difficulties picking up on sliders.

In [10]:
# Select all types of outs from Acuna's pitch by pitch data frame
acunaPbpOuts = acunaPbp[acunaPbp['events'].isin(pbpOutsSO)].copy()

# Show what pitches he's getting out the most on. 
print(acunaPbpOuts['pitch_name'].value_counts().to_string())

4-Seam Fastball    81
Slider             72
Sinker             52
Changeup           28
Cutter             28
Sweeper            26
Curveball          25
Split-Finger        3
Knuckle Curve       3
Other               1


# Drew Gilbert

Moving on to Drew Gilbert, most of the initial oberservations I made for Acuna can also be made for Gilbert. But it's important to note he's faced injury issues this year and has spent the least amount of time in the minors compared to his peers thus far. With that being said, most of his stats fall into the bottom quartile, besides his barrel ratios where he just just makes it into the second quartile. 

In [11]:
print(leaguewideAverages)
print(completeMetsAAAStats[defaultInfo + contactQuality].sort_values(by='Barrel/BBE%', ascending=False).iloc[2].to_string())

            BABIP   Hard Hit%    EV (MPH)  Adj. EV (MPH)   Dist (ft)      LA (°)  Barrel/BBE%  Barrel/PA%
count  471.000000  471.000000  471.000000     471.000000  471.000000  471.000000   471.000000  471.000000
mean     0.312605   35.555626   87.719108      91.218896  154.777070   13.382166     6.336518    3.971550
std      0.044797    8.125097    2.313921       0.818278   11.143512    4.839380     3.711165    2.194024
min      0.153000    9.400000   80.400000      89.200000  126.000000   -3.000000     0.000000    0.000000
25%      0.286000   30.600000   86.100000      90.600000  147.000000   10.000000     3.400000    2.300000
50%      0.314000   35.800000   87.700000      91.200000  155.000000   13.000000     5.700000    3.700000
75%      0.340000   41.450000   89.400000      91.700000  162.000000   17.000000     8.800000    5.400000
max      0.486000   59.100000   95.500000      94.100000  184.000000   26.000000    18.000000   10.700000
Player           Gilbert, Drew
Age            

Looking at Gilbert's outs by pitch, he shows a similar trend to Acuna except he struggles a bit more with the changeup than slider. 

In [12]:
gilbertPbp = metsAAApbpSavant[metsAAApbpSavant['player_name'] == 'Gilbert, Drew']
gilbertPbpOuts = gilbertPbp[gilbertPbp['events'].isin(pbpOutsSO)].copy()
print(gilbertPbpOuts['pitch_name'].value_counts().to_string())


4-Seam Fastball    28
Changeup           13
Slider             11
Sinker              7
Cutter              6
Curveball           3
Split-Finger        1


# Conclusions

After my research, I have come to some fairly straight forward conclusions. While Bretty Baty has been putting up decent numbers this season, they're still lower than his previous AAA stints indicating he might not have improved as much as hoped and he most likely see the results wanted if he were to get called up again. Some more time in AAA would definitely be beneficial for him because the shifts he's seeing will only get worse in the big leagues, especialy since defense is played at a much higher level as well.

Vientos was a great player to be called up this season. He showed significant improvement rather than regression in his most recent AAA stint and despite having low expected stats, he was making great, hard contact with the ball and consistently putting the ball into play. These are qualities that translate to major league talent and we can see that by how well Vientos has been peforming this season. 

While I don't think there were any plans to call up Luisangel Acuña or Drew Gilbert soon, they both need more time to acclimate to the AAA speed before they can go any higher. Acuña can get on base with his bat decently but he's making poor contact at the plate and seemingly struggling with a AAA level fastball. In the majors, his poor contact will most likely to weak outs rather than him beating out these balls like he is currently and the speed/movement of pitches will only get harder for him to hit. The same logic here applies for Gilbert; they could both benefit from another season of AAA. 

# Further Research

If I were to expand my research, I'd really be looking to compare this season's stats to how certain players peformed at lower levels in the minors. For example, due to Statcast not being available in AA, it's hard to look at how Acuña and Gilbert performed in single A and double A. I can look at season long stats to look at longer term treneds, but in this type of analysis the key is in the details. I'd also look at stats like runs created and advanced metrics related to plate disipline. Plate discipline is an essential part of baseball and is certainly a skill that can translate from the lower leagues to the majors. In addition, it would be important to analyze the minor league stats of players who have already been to the MLB and had success to better understand what stats should look like to see success in the MLB. This would also be much further down the road but implementing some level of model using Pytorch or Sci-Kit Learn to add more predictive data that's based on historical data I've collected and data from current players. 